In [4]:
!pip install small-text[transformers]==1.3.0

# setfit is an optional dependency and must be installed as well
!pip install setfit==0.5.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#!pip install datasets
from datasets import load_dataset

dataset = load_dataset('CarperAI/openai_summarize_comparisons')

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    valid2: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 50715
    })
    valid1: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 33082
    })
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 92534
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 83629
    })
})

In [7]:
test_dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='test[:100]')
train_dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='train[:1000]')

In [8]:
import random
random.seed(212)

In [9]:
import numpy as np
def preprocess_data(data):
  summaries = []
  labels = []
  for i in range(len(data)):
    a = np.random.choice(2)
    if a == 0:
      summaries.append(data[i]['chosen'])
    else:
      summaries.append(data[i]['rejected'])
    labels.append(a)
  return summaries, labels

In [10]:
"""def preprocess_data(data):
  summaries = []
  for i in range(len(data)):
    text1 = data["summaries"][i][0]['text']
    text2 = data["summaries"][i][1]['text']
    summaries.append([text1, text2])
  labels = data['choice']
  return summaries, labels"""

'def preprocess_data(data):\n  summaries = []\n  for i in range(len(data)):\n    text1 = data["summaries"][i][0][\'text\']\n    text2 = data["summaries"][i][1][\'text\']\n    summaries.append([text1, text2])\n  labels = data[\'choice\']\n  return summaries, labels'

In [11]:
train_summaries, train_labels = preprocess_data(train_dataset)

In [12]:
test_summaries, test_labels = preprocess_data(test_dataset)

In [14]:
from small_text import TextDataset
import numpy as np

num_classes = 2
target_labels = np.arange(num_classes)

train = TextDataset.from_arrays(train_summaries,
                                np.array(train_labels),
                                target_labels=target_labels)
test = TextDataset.from_arrays(test_summaries,
                               np.array(test_labels),
                               target_labels=target_labels)

/usr/local/lib/python3.9/dist-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


In [32]:
from small_text.integrations.transformers.classifiers.setfit import SetFitModelArguments
from small_text.integrations.transformers.classifiers.factories import SetFitClassificationFactory
from sentence_transformers.losses import ContrastiveLoss, CosineSimilarityLoss

num_classes = 2
sentence_transformer_model_name = 'sentence-transformers/all-mpnet-base-v2'
#sentence_transformer_model_name = "KikiQi/classification_AL"

setfit_model_args = SetFitModelArguments(sentence_transformer_model_name)
clf_factory = SetFitClassificationFactory(setfit_model_args, num_classes, )
                                          #classification_kwargs={'trainer_kwargs': {'num_epochs': 2, 'loss_class': CosineSimilarityLoss}})

In [37]:
from small_text import (
    PoolBasedActiveLearner, 
    random_initialization_balanced,
    BreakingTies,
    RandomSampling, 
    SubsamplingQueryStrategy
)

# define a query strategy and initialize a pool-based active learner
query_strategy = SubsamplingQueryStrategy(BreakingTies())   #不同策略改这里！！
# suppress progress bars in jupyter notebook
setfit_train_kwargs = {'show_progress_bar': True,}
active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train, fit_kwargs={'setfit_train_kwargs': setfit_train_kwargs})

# simulate a warm start
def initialize_active_learner(active_learner, y_train):

    #最开始有多少label(n_samples)
    x_indices_initial = random_initialization_balanced(y_train, n_samples=40)
    y_initial = y_train[x_indices_initial]
 
    active_learner.initialize_data(x_indices_initial, y_initial, indices_validation=np.arange(10))

    return x_indices_initial

initial_indices = initialize_active_learner(active_learner, train.y)
labeled_indices = initial_indices

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 1200
  Num epochs = 1
  Total optimization steps = 38
  Total train batch size = 32


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/38 [00:00<?, ?it/s]

In [38]:

import gc
import torch
import numpy as np
from sklearn.metrics import accuracy_score


num_queries = 5


def evaluate(active_learner, train, test):
    y_pred = active_learner.classifier.predict(train)
    y_pred_test = active_learner.classifier.predict(test)
    
    test_acc = accuracy_score(y_pred_test, test.y)

    print('Train accuracy: {:.2f}'.format(accuracy_score(y_pred, train.y)))
    print('Test accuracy: {:.2f}'.format(test_acc))
    
    return test_acc


results_setfit = []
results_setfit.append(evaluate(active_learner, train[labeled_indices], test))
    
for i in range(num_queries):
    # ...where each iteration consists of labelling 20 samples
    #每次加多少label(num_samples)
    q_indices = active_learner.query(num_samples=20)

    # Simulate user interaction here. Replace this for real-world usage.
    y = train.y[q_indices]

    # Return the labels for the current query to the active learner.
    active_learner.update(y, np.arange(5))
    
    # memory fix: https://github.com/UKPLab/sentence-transformers/issues/1793
    gc.collect()
    torch.cuda.empty_cache()

    labeled_indices = np.concatenate([q_indices, labeled_indices])

    print('---------------')
    print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
    results_setfit.append(evaluate(active_learner, train[labeled_indices], test))

print(labeled_indices)

Train accuracy: 0.88
Test accuracy: 0.54


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 2200
  Num epochs = 1
  Total optimization steps = 69
  Total train batch size = 32


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/69 [00:00<?, ?it/s]

---------------
Iteration #0 (60 samples)
Train accuracy: 1.00
Test accuracy: 0.58


...

---------------
Iteration #3 (120 samples)
Train accuracy: 0.96
Test accuracy: 0.63


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 5400
  Num epochs = 1
  Total optimization steps = 169
  Total train batch size = 32


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

---------------
Iteration #4 (140 samples)
Train accuracy: 0.94
Test accuracy: 0.65
[595 315 593 860  92 327 288 134 132 170 924 861 264 374 266 366 218 365
 370 362  85  53  65  71  75  81 238 242 148  47  87  45 465 895 892  37
  33 587  55  31 259 312 113 930 875 876 786 942 943 149 676 879 949 662
 468 171 469 932 243 234 699 597 898 131 332 596 495 719 334 698 901 685
 606 683 858 665 899 107 473 909  13 464 690 680 870 185 873 467  14 708
 707 110 684   0 904 670 669  94 409 411 466  22 928 428  21 538 590 405
  68 945 207 907 487 721 997 189 192 766 966 198 112 102 159 371 733 177
  82 999  73 249 291 967 793 450 647 585 219 548  30  66]


In [39]:
results_setfit

[0.54, 0.58, 0.66, 0.69, 0.63, 0.65]